# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [71]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [72]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [73]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [74]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [75]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [76]:
# TO-DO: Create a Keyspace 
session.execute('''
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    ''')

#### Set Keyspace

In [77]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session = cluster.connect('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [78]:
session.execute('''
    CREATE TABLE IF NOT EXISTS udacity.sessions_songs (
        sessionId     INT,
        itemInSession SMALLINT,
        artist        TEXT,
        length        DECIMAL,
        song          TEXT,
        PRIMARY KEY (sessionId, itemInSession)
    );
    ''')

In [79]:
session.execute('''
    CREATE TABLE IF NOT EXISTS udacity.user_sessions_songs (
        userId        INT,
        sessionId     INT,
        itemInSession SMALLINT,
        artist        TEXT,
        song          TEXT,
        firstName     TEXT,
        lastName      TEXT,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    );
    ''')

In [80]:
session.execute('''
    CREATE TABLE IF NOT EXISTS udacity.song_listeners (
        userId        INT,
        artist        TEXT,
        firstName     TEXT,
        lastName      TEXT,
        gender        TEXT,
        itemInSession SMALLINT,
        length        DECIMAL,
        level         TEXT,
        location      TEXT,
        sessionId     INT,
        song          TEXT,
        PRIMARY KEY (song, userId)
    );
    ''')

In [81]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO udacity.sessions_songs (sessionId, itemInSession, artist, length, song)"
        query = query + "VALUES (?, ?, ?, ?, ?)"
        query = session.prepare(query)
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (
                int(line[8]),   # sessionId
                int(line[3]),   # itemInSession
                line[0],        # artist
                float(line[5]), # length
                line[5]         # song
            )
        )

In [84]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO udacity.user_sessions_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (?, ?, ?, ?, ?, ?, ?)"
        query = session.prepare(query)
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (
                int(line[10]), # userId
                int(line[8]),  # sessionId
                int(line[3]),  # itemInSession
                line[0],       # artist
                line[9],       # song
                line[1],       # firstName
                line[4]        # lastName
            )
        )

In [87]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO udacity.song_listeners (userId, song, firstName, lastName, artist)"
        query = query + "VALUES (?, ?, ?, ?, ?)"
        query = session.prepare(query)
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (
                int(line[10]), # userId
                line[9],       # song
                line[1],       # firstName
                line[4],       # lastName
                line[0]        # artist
            )
        )

#### Do a SELECT to verify that the data have been inserted into each table

In [88]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
for row in session.execute('SELECT * FROM udacity.sessions_songs LIMIT 1'):
    print(f'sessions_songs: {row}')

for row in session.execute('SELECT * FROM udacity.user_sessions_songs LIMIT 1'):
    print(f'user_sessions_songs: {row}')

for row in session.execute('SELECT * FROM udacity.song_listeners LIMIT 1'):
    print(f'songs_listeners: {row}')

sessions_songs: Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=Decimal('191.08526000000000522049958817660808563232421875'), song='191.08526')
user_sessions_songs: Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', firstname='Emily', lastname='Benson', song='Sad Statue')
songs_listeners: Row(song="Wonder What's Next", userid=49, artist='Chevelle', firstname='Chloe', gender=None, iteminsession=None, lastname='Cuevas', length=None, level=None, location=None, sessionid=None)


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [89]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
result = session.execute('''
        SELECT
            artist,
            song,
            length
        FROM
            udacity.sessions_songs
        WHERE
            sessionid = 338
            AND iteminsession = 4;
    ''')

for row in result:
    print(row)

Row(artist='Faithless', song='495.3073', length=Decimal('495.30729999999999790816218592226505279541015625'))


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [90]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
result = session.execute('''
        SELECT
            artist,
            song,
            firstname,
            lastname
        FROM
            udacity.user_sessions_songs
        WHERE
            userid = 10
            AND sessionid = 182
        ORDER BY
            iteminsession;
    ''')

for row in result:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [92]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

result = session.execute('''
        SELECT
            userId,
            song,
            firstname,
            lastname
        FROM
            udacity.song_listeners
        WHERE
            song = 'All Hands Against His Own'
    ''')

for row in result:
    print(row)

Row(userid=29, song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(userid=80, song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(userid=95, song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [93]:
session.execute('DROP TABLE IF EXISTS udacity.sessions_songs')

In [94]:
session.execute('DROP TABLE IF EXISTS udacity.user_sessions_songs')

In [95]:
session.execute('DROP TABLE IF EXISTS udacity.song_listeners')

### Close the session and cluster connection¶

In [96]:
session.shutdown()
cluster.shutdown()